# Career Paths
____________________________________________________________________
__________________________________
Hey! If you're looking for a career path and want the computer to help you, this would be good for you.

If you're interested in AI, this is even better for you, since I'll be examining various AI-related methods to do so.

# Prolog Resolution
### Native Usage of Depth-First Backwards Chaining for Resolution
Prolog is a nice (and deceptively innocent-looking) programming language built for solving logical problems.
Let's start checking what prolog can do with its built-in solving resolution mehcanism.
Prolog's backend uses depth-first backwards chaining to perform unification. 
This means prolog starts from the goal and tries to find a way from the goal to the starting node. At each step, it tries to unify its temporary placeholder for step nelow, with the next available transition backwards from the goal to the starting point, until hitting the starting point.
Let's see how that will look like.

First, we'll make prolog into a little **expert system** which encodes many relevant roules about how to progress in the professional role. This usage of prolog is among its most native usage: chaining together many rules and naturally being able to resolve any possible conclusion through those rules. 

In [235]:
### These sections' code below is based on code from CS152 class 12.1 ###

KB_jobs = """
% Write your Prolog rules here

possible_future_job(programming) :- enjoying(programming), skilled_in(programming).
skilled_in(programming) :- experienced_in(programming).
skilled_in(computer_science) :- studying(computer_science); skilled_in(programming); skilled_in(data_science).
skilled_in(programming) :- studying(computer_science).

possible_future_job(product_management) :- enjoying(working_with_people), enjoying(project_management), studying(computer_science).
possible_future_job(product_management) :- skilled_in(programming), enjoying(project_management).

possible_future_job(data_science) :- enjoying(digging_through_data), studying(computer_science).

possible_future_job(computer_science_research) :- enjoying(research), studying(computer_science).

possible_future_job(computer_science_professor) :- enjoying(working_with_people), studying(computer_science).

possible_future_job(business) :- not(studying(computer_science)), 
                                (enjoying(working_with_people); enjoying(project_management)),
                                interested_in(a_high_paying_job).

possible_future_job(anything_you_like) :- not(interested_in(a_high_paying_job)) 
interested_in(X) :- studying(X). %assuming you are also interested in what you study 

"""

KB_asks = """
interested_in(X) :- ask(interested_in, X). 
skilled_in(X) :- ask(skilled_in, X).
experienced_in(X) :- ask(experienced_in, X).
studying(X) :- ask(studying, X).
enjoying(X) :- ask(enjoying, X).
"""

KB_prompt_ask = """
% The code below implements the prompting to ask the user:

% Asking clauses

ask(A, V):- %do we already know (A,V), no need to ask?
known(yes, A, V), % succeed if true
!. % stop looking (don't try any other ask in backtracking)

ask(A, V):-
known(_, A, V), % fail if false
!, fail.

% ask(A, V):- %A is not V if A is something else and could only be one thing
% not(multivalued(A)),
% known(yes, A, V2),
% V \== V2, %not equal ≠
% !, fail.

ask(A, V):-  %ask the user, succeed if user answers 'yes'
read_py(A,V,Y), % get the answer
asserta(known(Y, A, V)), % remember it
Y == yes. % succeed or fail
"""

KB = KB_jobs + KB_asks + KB_prompt_ask

with open("KB.pl", "w") as text_file:
    text_file.write(KB)

In [5]:
### START IMPORTING AND WORKING WITH PROLOG ###
### you should have the "pyswip" package installed through 
# pip install pyswip 
# or by manually downloading it to the same folder you are running this from.
# contact me if you need help with this.

### The code here will ask the user for input based on the askables
### It will check if the answer is known first
### This is not the interesting part for you but the setup. you can just trust this and run it.

import pyswip
from pyswip.prolog import Prolog
from pyswip.easy import *

prolog = Prolog() # Global handle to interpreter

retractall = Functor("retractall")
known = Functor("known",3)

# Define foreign functions for getting user input and writing to the screen
def write_py(X):
    print(str(X))
    sys.stdout.flush()
    return True

def read_py(A,V,Y):
    Y.unify(raw_input("Are you " + str(A) + " " + str(V) + "? -->"))
    return True


write_py.arity = 1
read_py.arity = 3

registerForeign(write_py)
registerForeign(read_py)
print("Prolog is ready, my friend.")

Prolog is ready, my friend.


In [7]:
### Get ideas from deterministic prolog background backwards chaining
prolog.consult("KB.pl") # open the KB
call(retractall(known))
print("Hey friend! Here are a few questions to get you some ideas for a future job\n(I mostly know now about computer scinece, sorry). \nAnswer with 'yes' or 'no' (or whatever else, but I'll take that as a no.")

for soln in prolog.query("possible_future_job(X).", maxresult=20):
    print("You could try working in " + soln['X'] +"\n")

print("I don't have any more ideas. \n If you want more ideas, consider asking a person who knows about this (or Google).")

Hey friend! Here are a few questions to get you some ideas for a future job
(I mostly know now about computer scinece, sorry). 
Answer with 'yes' or 'no' (or whatever else, but I'll take that as a no.
Are you enjoying programming? -->no
Are you enjoying working_with_people? -->yes
Are you enjoying project_management? -->no
Are you experienced_in programming? -->yes
Are you studying computer_science? -->yes
Are you skilled_in programming? -->yes
Are you enjoying digging_through_data? -->yes
You could try working in data_science

Are you enjoying research? -->yes
You could try working in computer_science_research

You could try working in computer_science_professor

I don't have any more ideas. 
 If you want more ideas, consider asking a person who knows about this (or Google).


# Forward Chaining or Backwards Chaining
Forward chaining will do the opposite: it will start looking from your starting point and build all the possiblities to proceed moving on, until (hopefully) reaching the goal in one of these paths.
Let's see what that would look like.


Below are a few more lessons about Forward Chaining taken from Russel and Norvig (2016).


### So What is Forward Chaining?
**Forward Chaining** is a type of *data-driven* reasoning. Meaning, reasoning in which the focus of attention starts with the known data. It can be used to keep building on top of the initial information we get (through the predefined "knowledge base", which you may think of as database for now, or through new inputs), derive conclusions from each piece of piece information, often without a specific query in mind.

That's why Forward Chaining is not the best for this purpose of *searching for a particular goal*. 

### We can make improvements to forward chaining

The forward-chaining algorithm shown before is designed for ease of understanding rather than for efficiency of operation. 
There are three possible sources of **inefficiency**. 

1. First, the “inner loop” of the algorithm involves finding a**ll possible unifiers** such that the premise of a rule unifies with a suitable set of facts in the knowledge base. This is often called **PATTERN MATCHING** and can be very expensive. 
2.  the algorithm **rechecks every rule on every iteration **to see whether its premises are satisfied, even if very few additions are made to the knowledge base on each iteration. 
3.  the algorithm might** generate many facts that are irrelevant** to the goal. 

There are then many variations to the forward chaining algorithm, for example: Incremental forward chaining, where it checks only for newly inferred facts at each stage; or RETE algorithm which preprocesses the set of rules in the knowledge base to construct a sort of dataflow network in which each node is a literal from a rule premise.



### Another improvement is to use Backwards Chaining in the face of Goal-Driven questions.

These algorithms work backward from the goal, chaining through rules to find known facts that support the proof.
Backward chaining implemented here is a **depth-first search algorithm**. Both Backward and Forward Chaining could be either depth-first, breadth-first, or some hybrid - depending on the implementation. Being Depth-First means that its space requirements are **linear in the size of the proof**.
It natively does so using backwards chaining for resolution.
This makes a lot of sense versus forward chaining, sense the queries we present it are **goal-driven**. Meaning, they do have a specific goal in mind. Usually, it is much more efficient if so to use the goal-driven backwards chaining. For much of the same reasons, prolog adpoted depth-first backwards chaining.

#### However, Prolog's use of depth-first backward chaining is still not perfect.
This algoithm is not perfect.
First, it might not be the most efficient in cases where the search trees are build like a pyramid shape: Consider a case where there is one goal, many many possible starting points deeply nested, and we look for a path from the start to the goal by backtracking from the goal, going through many branches excessively. In such cases, if we would start from the starting leaf node and forward chain up to the goal, we are almost sure to find it in minimal time. This is a general case that is less suitable for Backwards Chaining, but not really Prolog's fault. 
**However, a future suggestion for development, could be to internally represent the fact tree, and ACCORDING to the specific tree given, and the speciifc query given, choose the chaining algorithm used for resolution.**

More specifically, there is a mismatch between depth-first search and search trees that include *repeated states* and *infinite paths*. 
If the user generates a wrong recursion such as a recursive self-calling function without resolution (or "left-recursion"), prolog will fail. That's even if the necessary sentences appear right after. Therefore, Prolog is incomplete as a theorem prover.
Depth-first backward chaining also has problems with redundant computations.






According to Russel and Norvig's "Artificial Intelligence - A Modern Approach",
the pseudo-algorithm will look like this:

 
    function FOL-FC-ASK(KB,α) returns a substitution or false 
    
        # inputs: KB , the knowledge base, 
                a set of first-order definite clauses
                α, the query, an atomic sentence
        # local variables: new (the new sentences 
                           inferred on each iteration)

        repeat until new is empty 
            new ← { }
            for each rule in KB do
                (p1 ∧ . . . ∧ pn ⇒ q) ← 
                              STANDARDIZE-VARIABLES(rule)
                    for each θ such that SUBST(θ,p1 ∧ ... ∧ pn) = 
                               SUBST(θ,p1′ ∧ ... ∧ pn′ ) 
                               for some p1′ ,...,pn′ in KB 
                q′ ←SUBST(θ,q)
                if q′ does not unify with some sentence already in KB or new then 
                    add q′ to new
                    φ ← UNIFY(q′, α)
                    if φ is not fail then return φ 
            add new to KB
        return false


*Source: Russell, S. J., & Norvig, P. (2016). Artificial intelligence: a modern approach (3rd ed.). Boston: Pearson.*
            
#### Below I will implement a simple depth-first forward-chaining algorithm.
For this problem, the tree is much wider then it is deep. Meaning, there are many short paths, instead of few deep paths. If the saught-for goal hides within the paths stemming of one of the first lines, depth-first search is more efficient. 

In [136]:
# PROLOG CODE

KB = """
% building a path recursive function. 
% recursive functions in prolog need an initialization function
% and the function that will call the recursion afterwards

forward_chain(A, A, _, [A]). %Stopping condition: This is when to stop: when start, goal, and start is all the same (when we've reached the goal).
forward_chain(Start, Goal, Visited_List, [Start | Path_List]) :-   
     trans(Start, X), %first step
     not(member(X, Visited_List)), %check that we haven't visited that node already. we don't want to repeat.
     forward_chain(X, Goal, [X | Visited_List], Path_List). % recursive call to path with the next job, also adding X to visited list

backward_chain(A, A, _, [A]).  %Stopping condition: This is when to stop: when start, goal, and start is all the same (when we've reached the goal).          
backward_chain(Start, Goal, Visited, [Goal | Path_List]) :-   
        trans(X, Goal),     % first step
        not(member(X, Visited)), % check that we haven't visited that node already. we don't want to repeat.
        backward_chain(Start, X, [X | Visited], Path_List). % recursive call to path with the next job, while also adding X to visited list



% below we list the possilbe transitions (trans(Job1,Job2))
% (notice that the transitions go from left to right)

% POSITIONS LIST START %
trans(business_job , ceo ).
trans(cto , ceo ).
trans(starting_your_own_startup , ceo ).
trans(having_business_skils , starting_your_own_startup ).
trans(computer_science_job , starting_your_own_startup ).
trans(starting_your_own_startup , cto ).
trans(programmer , cto ).
trans(data_science , cto ).
trans(computer_science_job , cto ).
trans(studying_computer_science , programmer ).
trans(studying_computer_science , data_science ).
trans(computer_science_internship , programmer ).
trans(computer_science_internship , data_science ).
trans(data_science , computer_science_job ).
trans(computer_science , computer_science_research ).
trans(computer_science , computer_science_professor ).
trans(studying_computer_science , computer_science_internship ).
trans(studying_computer_science , computer_science_job ).
trans(studying_business , business_job ).
trans(having_business_skils , business_job ).
trans(business_internship , business_job ).
% END %
"""

with open("KB.pl", "w") as text_file:
    text_file.write(KB)

In [138]:
# extract states list from KB
# This is not interesting for you but for internal usage

def extract_states(KB, start_sign = 'START %\n', end_sign = '\n% END %'):
    """ This function extracts the states from the KB, 
        including duplicates, to a list. I'm not using a set because 
        later I would want to use the count of duplicates"""
    #start_sign = 'START %\n'
    #end_sign = '\n% END %'
    lines = KB.split(start_sign)[-1].split(end_sign)[0] #take only the restaurant lines between the starting and ending points
    lines = lines.splitlines() #now we have a list of restaurants

    # CREATE RESTAURANTS/ATTRIBUTES TABLE OUT OF TEXT KB
    states_list = []
    for line in lines:
        #extract the attributes and strip space if there are, for safety
        states = line.split("(")[-1].split(")")[0].replace(' ', '')  
        states = states.split(",") #make a list of the attributes    
        states_list += states
    return states_list


def extract_unique_states_set(KB):
    """ Making into a unique set"""
    positions = extract_states(KB)
    unique_positions = (set(positions))
    return unique_positions

def print_states(KB):
    """ printing the unique set"""
    states_set = extract_unique_states_set(KB)
    for i in states_set:        
        print i


In [162]:
### Get ideas from deterministic prolog background backwards chaining
### I INTENTIONALLY MAKE THE USER TYPE IN ANSWERS RATHER THAN NUMBERED MENU,
### I BELIEVE IT IS MORE "NATURAL LANGUAGE" FEEL THAT I WANT TO GIVE TO THE USER,
### MORE INTUITIVE, AND MAKES SURE THAT THE USER DIDN'T INSERT A WRONG NUMBER BY MISTAKE.



def opener_explore_jobs(KB=KB,mode="chaining",jobs_list=None):
    if mode=="chaining":
        print("Hey! Here you can explore possible job transitions, according to what I know.") 
    
    #later I will use these openings:
    if mode=="prob":
        print("Hey! Here you can explore possible job transitions PROBABILITIES, according to what I know.")
    if mode=="shortest_path":
        print("Now you can find the most likely path from starting position to your goal.")
    
    # print positions
    print("\nBelow are the jobs I have some information on:\n")
    if jobs_list == None: #Later I'll just want to insert that directly:
        print_states(KB)
    else:
        for i in jobs_list: print i
    
    
def display_paths_forward_chaining(query_str,printquery=True,limit=20):
    """#### PRETTY DISPLAY SOLUTION PATH #### 
         limit: the limit number of results to display
         printquery: whether you don't want to display also the query being called in prolog
    """
    counter = 1
    for soln in prolog.query(query_str, maxresult=limit):
        if printquery: print(query_str)
        print("\nPotential Career Path #{}".format(counter))
        solution = soln['Path_List']
        for i in range(len(solution)):
            print("{}. {}".format(i+1,solution[i]))
        counter += 1
        if counter==limit:
            print("Alright, that's too many options already. If you want to see more, edit my 'maxresults' arguement, or press ';'")
            if raw_input("more?") in ['yes','Y',';']:
                display_paths_forward_chaining(query_str=';') #recurse with ";"

    
def explore_KB_graph(mode="forward",printquery=True):
    """ DIALOGUE with user to explore the KB career path graph
        mode: default, "forward" for forward chaining.
            if you want to use backwards chaining, chaing the mode to 'backward' """

    prolog.consult("KB.pl") # open the KB
    call(retractall(known))

    start = raw_input("Enter a starting position you want to explore from: ")
    goal = raw_input("Enter a goal job that you'd want to reach: ")
    
    # using forward chaining
    query_str = "forward_chain({start}, {goal}, [{visited}], Path_List) .".format(start=start,goal=goal,visited=start) #Path List is just a variable to initialize that list with nothing real in it
    
    if mode == "backward":
        #using backwards chaining
        query_str = "backward_chain({}, {}, [{}], Path_List) .".format(start,goal,start)
    
    #### PRETTY DISPLAY SOLUTION PATH ####
    display_paths_forward_chaining(query_str,printquery=True)

    print("\n\nI don't have any more ideas. \n If you want more ideas, consider asking a person who knows about this (or Google).")
    return
    
opener_explore_jobs() 


Hey! Here you can explore possible job transitions, according to what I know.

Below are the jobs I have some information on:

computer_science
studying_business
ceo
studying_computer_science
having_business_skils
starting_your_own_startup
data_science
computer_science_research
computer_science_internship
computer_science_job
programmer
cto
computer_science_professor
business_internship
business_job


In [80]:
explore_KB_graph(mode="forward",printquery=True)

Enter a starting position you want to explore from: studying_computer_science
Enter a goal job that you'd want to reach: ceo
forward_chain(studying_computer_science, ceo, [studying_computer_science], Path_List) .

Potential Career Path #1
1. studying_computer_science
2. programmer
3. cto
4. ceo
forward_chain(studying_computer_science, ceo, [studying_computer_science], Path_List) .

Potential Career Path #2
1. studying_computer_science
2. data_science
3. cto
4. ceo
forward_chain(studying_computer_science, ceo, [studying_computer_science], Path_List) .

Potential Career Path #3
1. studying_computer_science
2. data_science
3. computer_science_job
4. starting_your_own_startup
5. ceo
forward_chain(studying_computer_science, ceo, [studying_computer_science], Path_List) .

Potential Career Path #4
1. studying_computer_science
2. data_science
3. computer_science_job
4. cto
5. ceo
forward_chain(studying_computer_science, ceo, [studying_computer_science], Path_List) .

Potential Career Path #5
1

**For these purposes, backwards chaining might be a better usage if you have a goal in mind. The best usage for having both forward and backward chaining algorithms here is in enabling us to:**
1. Use Forward Chaining to explore *any possible path* from a given starting point. For example, if you don't have a goal, and want to see what are the known likely paths to pursue from your current experience/position, you could use forward chaining algorithm to build paths from your starting point onwards and present all the possible paths from this starting point.
2. Converstly, Use Backward Chaining in order to do the opposite. If you have a goal in mind, and you just want to see *from where could you start to reach that goal and taking which paths*, this is what you should use.
3. If you have a goal - you should use backwards chaining anyway, since it is more efficient for these type of goal-oriented searches - especially since in these dataset we'd have more connections at the bottom and less at the top so it's good to be goal-driven.

All you need to do is to modify the running line above from:
explore_KB_graph(mode="forward") #forward chaining
explore_KB_graph(mode="backward") #backward chaining

If you don't have a starting/goal position, when prolog asks you - type "X" or anything starting a captial letter. This will make prolog interpret it as a Variable that could mean "anything", instead of the specific known name like "ceo".

# Probabilistic Graphs and interpretations

#### But life isn't deterministic. Certainly not career paths and your future. 
##### Let's model this with probabilities instead of deterministic logic.

Frankly, it doesn't really make sense to say that if you'll study computer science you'll definitely be a programmer, but we want to model it with the probability of you getting from studying computer science to being a programmer.

For that, I'll use a variant of prolog called "Problog". It is as intuitive and simple, but built for probabilistic calculations.
Therefore, we can easily define all the edges of a graph and their probability as a weight. Problog will automatically will compute the probability of there being a path between them.

You can install prolog directly into python from your terminal with:
        
        pip install problog
        
Then you can run it from python by importing the necessary commands.
Read the documentation here:
http://problog.readthedocs.io/en/latest/index.html

How does it calculate? The definition was not to be found straightforwardly online. However, according to the information I could gather and experiments, I understand the following.
This calculation below will show the cumulative probability of reaching from A to B, by the follwoing backend calculation:
1. A possible path's probabilities are multiplied by each other, as a joint probability. For example: data_science->cto = 0.5 and cto->ceo = 0.9, so getting form data_science to ceo will be calculated as 0.5\*0.9 = 0.45.
2. Each additional path to get from the same start to the end is then added on top of that, but taking into account that you chose the "other way" - so at each "choice point" that you must have chosen the other way, it detracts the complement of the probability and multiplies by that; such that probabilities are not added to be more than 1 ever.

Understanding this was a joint product of reading this: Jansen, 2012. "Study of Most Probable Shortest Path Problems in ProbLog". *KU Leuven*. Retrieved from: 
https://people.cs.kuleuven.be/~theofrastos.mantadelis/appendixs/Joachim_MasterThesis.pdf 
and proving to myself using these calculations below. 

iterating through each step (let's say from the start to the goal as in forward chaining for explanation), and multiplying the probability by the amount that is left



In [140]:

problog_transitions = """
% POSITIONS LIST START %
0.9::trans(business_job , ceo ).
0.9::trans(cto , ceo ).
0.7::trans(starting_your_own_startup , ceo ).
0.4::trans(having_business_skils , starting_your_own_startup ).
0.5::trans(computer_science_job , starting_your_own_startup ).
0.9::trans(programmer , cto ).
0.5::trans(data_science , cto ).
0.8::trans(computer_science_job , cto ).
0.8::trans(studying_computer_science , programmer ).
0.5::trans(studying_computer_science , data_science ).
0.7::trans(computer_science_internship , programmer ).
0.5::trans(computer_science_internship , data_science ).
0.6::trans(data_science , computer_science_job ).
0.9::trans(computer_science , computer_science_research ).
0.8::trans(computer_science , computer_science_professor ).
0.5::trans(studying_computer_science , computer_science_internship ).
0.8::trans(studying_computer_science , computer_science_job ).
0.5::trans(studying_business , business_job ).
0.7::trans(having_business_skils , business_job ).
0.8::trans(business_internship , business_job ).
% END %
"""

# SIMPLE Path finder by a recursive function. No big frills here. 
# Like a simplified forward chaining algorithm we had before
problog_path_finder = """
path(X,Y) :- trans(X,Y).
path(X,Y) :- trans(X,Z),
             Y \== Z,
         path(Z,Y).
"""

### DFEINING DIALOGUE FUNCTIONS 

opener_explore_jobs(KB=problog_transitions,mode="prob")

Hey! Here you can explore possible job transitions PROBABILITIES, according to what I know.

Below are the jobs I have some information on:

computer_science
studying_business
ceo
studying_computer_science
having_business_skils
starting_your_own_startup
data_science
computer_science_research
computer_science_internship
computer_science_job
programmer
cto
computer_science_professor
business_internship
business_job


In [177]:
def prob_dialogue1(problog_transitions,problog_path_finder):
    starting_positions = raw_input("Which positions would you be interested to start from? \nplease insert as list of names without spaces, as in: \nbusiness_internship,studying_business,studying_computer_science\n")
    starting_positions = starting_positions.split(',')

    end_positions = raw_input("Which positions interest you as future goals? \nplease insert as list of names without spaces, as in: \ncto,ceo,business_job\n")
    end_positions = end_positions.split(',')

    queries = []
    for start in starting_positions:
        for goal in end_positions:
            query = "query(path({start_str},{goal_str})).".format(start_str=start, goal_str=goal)
            queries.append(query)

    queries_text = "\n".join(queries) #make list into a text list with lines

    ProblogModel = PrologString(problog_transitions + problog_path_finder + queries_text) # put together to a model we'll run in Problpg

    result = get_evaluatable().create_from(ProblogModel).evaluate()
    print("\nHere is all I could find \nand my estimated probabilities for getting from these starting positions to the goals:\n")
    return result


result = prob_dialogue1(problog_transitions,problog_path_finder)
result

Which positions would you be interested to start from? 
please insert as list of names without spaces, as in: 
business_internship,studying_business,studying_computer_science
business_internship,studying_business,studying_computer_science
Which positions interest you as future goals? 
please insert as list of names without spaces, as in: 
cto,ceo,business_job
cto,ceo,business_job

Here is all I could find 
and my estimated probabilities for getting from these starting positions to the goals:



{path(business_internship,cto): 0.0,
 path(business_internship,ceo): 0.7200000000000001,
 path(business_internship,business_job): 0.8,
 path(studying_business,cto): 0.0,
 path(studying_business,ceo): 0.45000000000000007,
 path(studying_business,business_job): 0.5,
 path(studying_computer_science,cto): 0.951007,
 path(studying_computer_science,ceo): 0.894731695,
 path(studying_computer_science,business_job): 0.0}

# SEARCHING FOR THE BEST PATH

So far, we've seen simple determinsitc chaining algorithms, then improved by a probabilistic interpretation that allowed you to see all the probabilities to get from each start to each end. 
But now, maybe after you have explored the probabilities for getting from A to B and you decide you'd like to try one out, which career path should you aim for?

Here, I'll assume that you'd want to take the "safest" route.
By "Safest", I mean the one with the highest probability.
I'm working on collecting some data from Linkedin, which I could later analyze and get the *relative frequencies* of each job type transition in the whole dataset as the probabilities. 
The data collection took much more time than expected, so for now this will be implemented with toy sample data as before.  But it could later be easily extended.
You can see the sample data here: https://docs.google.com/spreadsheets/d/1jVDpHkjsyJgPnpqSuuyU1bbbGND8exoifI3C7duedPg/edit#gid=0

#### Short Solution #1
We already got a list of potential paths and their probabilities. 
We can apply the same algorithm and at the end just summarize what is the most likely path. 

In [197]:
print("From what you've told us before, the 'safest' (most likely) career path between your potential paths would be:")
best_path = max(result, key=result.get)
print("Start and end positions: {}. \nLikelihood of transition: {}".format(str(best_path)[4:],result[best_path]))

From what you've told us before, the 'safest' (most likely) career path between your potential paths would be:
Start and end positions: (studying_computer_science,cto). 
Likelihood of transition: 0.951007


#### Solution #2:
##  A Built-in Depth-First Graph Search for the Shortest Path
The above solution was a no-brainer, it simply "brute-forced" all solutions then picked the best one. It is more akin to a breadth-first search, but rather than inspecting the entire first layer and then proceeding to the best next step, it inspects ALL THE POSSIBLE PATHS first, then chooses. (So not really like a breadth first, but if the "first step" was calculating the entire path down each road...). 
It is not likely to get the best performance in large datasets, although it is certainly complete and sound (it will always find the best solution if solutions exist).

So to solve this problem in a more native way, I'll make a WEIGHTED GRAPH for us to perform a WEIGHTED GRAPH SEARCH, where the weights correspond to the probabilities - but in their inverse, since the algorithms want to MINIMIZE the cost. This is a Depth-First Search, looking down at each branch until hitting the solution or backtracking to the latest intersection to inspect the other direction it missed most recently. 

This is a basic search algorithm that could be improved too, but for the size of our KB and purpose that would be an unnecessary overkill. 
If you are interested, look up some improvmenet for Depth First algorithms such as:

1. Depth-limited search, where the search algorithm is limited to a depth level ("e.g., only go down 20 levels; i don't want to have more than 20 job transitions to reach my goal anyway). This is important to stop Depth First search from looping between sub-loops of mutually connected nodes infinitely, if there are any. There aren't any in our KB. But still this is not the ideal solution.
2. Iterative Deepening depth-first search -  this algorithm finds the best depth limit. It does this by gradually increasing the limit—first, from 0, then 1, then 2, and so on—until a goal is found. At each stage it will check if it could reach the solution at this depth. This will occur when the depth limit reaches d, the depth of the shallowest goal node. Meaning, *it will only find the Shallowest Solution*. In our later implementation of finding the minimal cost path in a weighted graphs, that could be misleading, since the weights of (A,B,C) could be lower in total than the jump from A to C. We will apply it to have the "safest" (risk free) job routes; so moving from a programmer to a CTO to a CEO is "safer" than the leap straight to CEO.
3. Bidirectional Search - this would be running two simultaneous searches—one forward from the initial state and the other backward from the goal—hoping that the two searches meet in the middle. Kind of like running the forward chaining and backward chaining until they meet, but using a particular kind of each.



It is important to note that in this tutorial and applications our goal is to implement **simple and (reltively) understandable solutions** rather than the most sophisticated algorithms, since they are (a) not needed otherwise for this purpose and (B) the most relevant for the #audience.


In [199]:
#first, translating problog transitions to a prolog style predicate of airity 3: Start, End, Weight.

# since we'd want to minimize weights, I would translate only for computability 
# the probabilities to their complement (1-probability), in this version.
# we don't want to maximize since that could lead to loops and redundant paths
# we could then think about the transitions probabilities as: 
# "Chances NOT TO GET JOB B AFTER JOB A", which we'd want to minimize.
def prob_complement(prob):
    return 1 - float(prob)

def translate_prob_to_prol(KB, weight_conversion, start_sign = 'START %\n', end_sign = '\n% END %'):
    lines = KB.split(start_sign)[-1].split(end_sign)[0] #take only the restaurant lines between the starting and ending points
    lines = lines.splitlines() #now we have a list of restaurants

    # CREATE RESTAURANTS/ATTRIBUTES TABLE OUT OF TEXT KB
    new_lines = []
    jobs_set = set([])
    for line in lines:
        #extract the attributes and strip space if there are, for safety
        weight = line.split("::")[0] #get the weight
        weight = weight_conversion(weight) #convert the weight according to the function of choice
        #print weight
        jobs = line.split("(")[-1].split(")")[0].replace(' ', '')  
        jobs = jobs.split(",") #make a list of the attributes    
        
        for job in jobs:
            jobs_set.add(job)
        
        newline = "trans({}, {}, {}).".format(jobs[0],jobs[1],weight)
        new_lines.append(newline)
    new_lines_as_text_chunk = "\n".join(new_lines)
    return new_lines_as_text_chunk, jobs_set

min_transitions,jobs_set = translate_prob_to_prol(KB = problog_transitions, weight_conversion = prob_complement)

In [225]:
shortest_path_prolog = """
%%% SHORTEST PATH PROLOG %%%

% recusrive definition for finding a single path, similar to before, but with weights.
single_path(Start, Goal, Weight, [Start,Goal], _) :- trans(Start, Goal, Weight).  %stopping condition: direct transition between Start and Goal
single_path(Start, Goal, Weight, [Start|Path], Visited) :-  not(member(Start, Visited)), %first check that this next node was not visited already
                                 trans(Start, Middle, Weight1),
                                 single_path(Middle, Goal, Weight2, Path, [Start|Visited]),
                                 Weight is Weight1 + Weight2.
            
% Defining a Dynamic Predicate for the Solution. 
% (Initializing with "If [nothing] :- something" makes this entailment always true.)
%import(dynamic).
%:- dynamic(path_found/2).
%Starting with the single shortest path - as the "stopping condition" predicate:
shortest_path(Start, Goal, Weight, Path) :- \+ path_found(_, _),  
                           single_path(Start, Goal, Weight1, Path1, []),
                           assertz(path_found(Weight1, Path1)), %add new fact to bottom (%z)
                           !,
                           shortest_path(Start,Goal,Weight,Path).

% Defining a recursive predicate for the shortest path, with depth-first search
shortest_path(Start, Goal, _, _) :- single_path(Start, Goal, Weight1, Path1, []), 
                           path_found(Weight2, Path2), Weight1 < Weight2,
                           retract(path_found(Weight2, Path2)), %backtracking to retrieve the path from the lastest branching points.
                           asserta(path_found(Weight1, Path1)), %when a new path is found, add it to the KB at the top
                           fail. %if finished, "fail" to finish this predicate.

% retrieveing the final solution of the shortest path
% %backtracking to retrieve the path from the lastest branching points.
shortest_path(_, _, Weight, Path) :- path_found(Weight,Path), retract(path_found(Weight,Path)). 

"""
min_transitions_text = " % START %\n" + min_transitions + "% END %"
KB_shortest_path = shortest_path_prolog + min_transitions_text

opener_explore_jobs(KB=KB_shortest_path, mode="shortest_path", jobs_list=jobs_set)

Now you can find the most likely path from starting position to your goal.

Below are the jobs I have some information on:

computer_science
studying_business
ceo
studying_computer_science
having_business_skils
starting_your_own_startup
data_science
computer_science_research
computer_science_internship
computer_science_job
programmer
cto
computer_science_professor
business_internship
business_job


In [228]:
def shortest_1path_dialogue(KB):
    print("Here I'll find the 'safest' (most likely) career path between your desired start and end positions")
    start = raw_input("Which position would you be interested to start from?")
    goal = raw_input("Which job is your future goal?")

    query = "query(shortest_path({start_str},{goal_str},Weight,Path_List)).".format(start_str=start, goal_str=goal)
    #query = "shortest_path(computer_science_internship,ceo,W,P)." #DEBUG
    ProblogModel = PrologString(KB + query) # put together to a model we'll run in Problpg

    result = get_evaluatable().create_from(ProblogModel).evaluate()
    print("\nHere is all I could find \nand my estimated probabilities for getting from these starting positions to the goals:\n")
    return result

shortest_1path_dialogue(KB_shortest_path)

Here I'll find the 'safest' (most likely) career path between your desired start and end positions
Which position would you be interested to start from?ceo
Which job is your future goal?cto

Here is all I could find 
and my estimated probabilities for getting from these starting positions to the goals:



{}

** Sorry guys. There seems to be currently a problem with python to implement the "Dynamic" predicate through Pywsip which I couldn't solve in time. But this IS the right procedure and syntax. I've used it in a standalone Prolog shell at https://swish.swi-prolog.org/.
I'll paste in the results here. 
If you're interested, I'll also paste the prolog code here below, with the transitions ready, so you can copy-paste it and run it yourself there. **

### PLEASE SEE RESULTS OF THE SAFEST PATH ALGORITHM AT THE END.

In [234]:
#### YOU CAN PASTE THIS TEXT BELOW (between the """ ... """) into: https://swish.swi-prolog.org/.
#### Then query as in: 
# shortest_path(computer_science_internship,ceo,Risk_Level,Path).
# without the "#" but with the "." at the end!

KB_shortest_path = """

% recusrive definition for finding a single path, similar to before, but with weights.
single_path(Start, Goal, Weight, [Start,Goal], _) :- trans(Start, Goal, Weight).  %stopping condition: direct transition between Start and Goal
single_path(Start, Goal, Weight, [Start|Path], Visited) :-  not(member(Start, Visited)), %first check that this next node was not visited already
                                 trans(Start, Middle, Weight1),
                                 single_path(Middle, Goal, Weight2, Path, [Start|Visited]),
                                 Weight is Weight1 + Weight2.
            
% Defining a Dynamic Predicate for the Solution. 
% (Initializing with "If [nothing] :- something" makes this entailment always true.)
import(dynamic).
:- dynamic(path_found/2).

%Starting with the single shortest path - as the "stopping condition" predicate:
shortest_path(Start, Goal, Weight, Path) :- \+ path_found(_, _),  
                           single_path(Start, Goal, Weight1, Path1, []),
                           assertz(path_found(Weight1, Path1)), %add new fact to bottom (%z)
                           !,
                           shortest_path(Start,Goal,Weight,Path).

% Defining a recursive predicate for the shortest path, with depth-first search
shortest_path(Start, Goal, _, _) :- single_path(Start, Goal, Weight1, Path1, []), 
                           path_found(Weight2, Path2), Weight1 < Weight2,
                           retract(path_found(Weight2, Path2)), %backtracking to retrieve the path from the lastest branching points.
                           asserta(path_found(Weight1, Path1)), %when a new path is found, add it to the KB at the top
                           fail. %if finished, "fail" to finish this predicate.

% retrieveing the final solution of the shortest path
% %backtracking to retrieve the path from the lastest branching points.
shortest_path(_, _, Weight, Path) :- path_found(Weight,Path), retract(path_found(Weight,Path)). 

trans(business_job, ceo, 0.1).
trans(cto, ceo, 0.1).
trans(starting_your_own_startup, ceo, 0.3).
trans(having_business_skils, starting_your_own_startup, 0.6).
trans(computer_science_job, starting_your_own_startup, 0.5).
trans(programmer, cto, 0.1).
trans(data_science, cto, 0.5).
trans(computer_science_job, cto, 0.2).
trans(studying_computer_science, programmer, 0.2).
trans(studying_computer_science, data_science, 0.5).
trans(computer_science_internship, programmer, 0.3).
trans(computer_science_internship, data_science, 0.5).
trans(data_science, computer_science_job, 0.4).
trans(computer_science, computer_science_research, 0.1).
trans(computer_science, computer_science_professor, 0.2).
trans(studying_computer_science, computer_science_internship, 0.5).
trans(studying_computer_science, computer_science_job, 0.2).
trans(studying_business, business_job, 0.5).
trans(having_business_skils, business_job, 0.3).
trans(business_internship, business_job, 0.2).
"""



## Appendix to the Appendix
#### PROBABILISTIC DECISION MAKER AGENT ####

I will leave this out for now, since it didn't work when running from Python (and I'm running out of time). But it's a nice direction worth pursuing later.

In [108]:
#### PROBABILISTIC DECISION MAKER AGENT ####
#### Code adapted from https://dtai.cs.kuleuven.be/problog ###

from problog.tasks.dtproblog import dtproblog
from problog.program import PrologString
""" Sometimes just having someone asking you these questions 
    one by one and telling you what to do is all you need.
    Especially in finals. 
    This model can decide for you.
    Change the probabilities of acceptance to the companies or types of companies that you want
    and the decisions you want the program to make for you"""

model = """
    % Probabilities of being accepted.  %
    % CHANGE TO WHAT IS RELEVANT FOR YOU, as probabilities between 0-1. %
    0.1::accepted_tech_giant.
    0.3::accepted_tech_medium_company.
    0.5::accepted_tech_startup.
    0.7::accepted_nontech
    

    %these look like deterministic rules, but they will factor in the probabilities I listed above
    accepted_tech_giant :- apply_to_tech_giant.
    accepted_tech_medium_company :- apply_to_tech_medium_company.
    accepted_tech_startup :- apply_to_tech_startup.
    accepted_tech :- accepted_tech_giant; accepted_tech_medium_company; accepted_tech_startup.
    apply_tech :- apply_to_tech_startup; apply_to_tech_medium_company; apply_to_tech_giant.
    accpeted_nontech :- apply_to_nontech.
    accpeted_anything :- accepted_tech ; accpeted_nontech.
    
    
    % Decisions of where to apply to
    %?::apply_to_tech_giant.
    %?::apply_to_tech_medium_company.
    ?::apply_tech.
    ?::apply_to_nontech.
    
    
    % CHANGE YOUR UTILITIES HERE %
    % Order them on a continuous scale. You can do from -100 to +100 for example
    % Let's say I prefer a tech giant, tech in general ,but not having anything is the worst%
    utility(accepted_tech_giant, 80).
    utility(accepted_tech_medium_company, 50).
    utility(accepted_tech_startup, 30).
    utility(accepted_tech, 30).
    utility(accpeted_nontech, 20).
    utility(not(accpeted_anything), -100).
"""

#program = PrologString(model)
#decisions, score, statistics = dtproblog(program)

#for name, value in decisions.items():
#    print ('%s: %s' % (name, value))
""" I will leave this out for now, since it didn't work when running from Python (and I'm running out of time). But it's a nice direction worth pursuing later."""

" I will leave this out for now, since it didn't work when running from Python (and I'm running out of time). But it's a nice direction worth pursuing later."

### References

*Russell, S. J., & Norvig, P. (2016). Artificial intelligence: a modern approach (3rd ed.). Boston: Pearson.*

*ProbLog 2.1 documentation (2017). Retrieved From: http://problog.readthedocs.io/en/latest/index.html*

*Problog (2017). KU Leuven · DTAI Research Group. https://dtai.cs.kuleuven.be/problog/index.html*

*Jansen, 2012. "Study of Most Probable Shortest Path Problems in ProbLog". *KU Leuven*. Retrieved from: 
https://people.cs.kuleuven.be/~theofrastos.mantadelis/appendixs/Joachim_MasterThesis.pdf *

*Prolog and Graphs, Rodny's Corner. https://rlgomes.github.io/work/prolog/2012/05/22/19.00-prolog-and-graphs.html*

*Code from Class 12.1*

*Code from LBA Assignment made by me.*